In [1]:
# https://nlp.johnsnowlabs.com/docs/en/annotators#languagedetectordl

# maven - com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.3
# pypi - spark-nlp, vaderSentiment, boto3==1.9.157

# example on sentiment analysis
  # https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/SentimentDL_train_multiclass_sentiment_classifier.ipynb
# https://nlp.johnsnowlabs.com/docs/en/pipelines#multi-language

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark import SparkFiles
from pyspark.ml.feature import (
    StringIndexer,
    HashingTF, 
    IDF
)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes

In [3]:
# get or create Spark session

app_name = "news_analysis"

spark = SparkSession.builder.appName(app_name) \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.3") \
    .getOrCreate()

In [4]:
file_uri = "/mnt/mnt_s3/BusinessTrainData.csv"

trainDataset = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(file_uri)
display(trainDataset)

_c0,index,category,headline,authors,link,short_description,date,short_description_new,label
148341,170325,BUSINESS,"Stores Open On Christmas Eve Include Walmart, Best Buy, Target, Among Others",Caroline Fairchild,https://www.huffingtonpost.com/entry/stores-open-on-christmas-eve-2012_us_5bb2e167e4b0480ca65de749,leave christmas shopping minute worry americas retailers got covered,2012-12-22,leave christmas shopping minute worry americas retailers got covered,Negative
94803,106274,BUSINESS,Trusting the Crowd and the Machines,"Despina Katsikakis, ContributorGlobally renowned expert on how to transform the workplace to ...",https://www.huffingtonpost.com/entry/trusting-the-crowd-and-th_b_6166406.html,business environment future needs trust people technology provide flexibility choice employees connect complementary skills network work challenges learn fast unlock passion improve performance,2014-11-19,business environment future needs trust people technology provide flexibility choice employees connect complementary skills network work challenges learn fast unlock passion improve performance,Positive
86152,90632,BUSINESS,"""The Sudden Implosion of """"Secret"""" May Reveal A Secret: Is Silicon Valley Out Of Touch Or Just Losing It's Touch?""","Tony Potts, ContributorPartner-SierraMaya360/PartnersTrust/Prev-Access Hollywood",https://www.huffingtonpost.com/entry/the-sudden-implosion-of-s_b_7310388.html,look secret timeline demise started august inverted hockey stick trajectory came days raised july investors nightmare doubt,2015-05-19,look secret timeline demise started august inverted hockey stick trajectory came days raised july investors nightmare doubt,Negative
40379,41278,BUSINESS,He's Grounded! Delta Bans Obnoxious Trump Supporter For Life,Carla Herreria,https://www.huffingtonpost.com/entry/delta-bans-trump-supporter_us_583ce595e4b0860d611640a2,ceo says passengers flight given refunds,2016-11-29,ceo says passengers flight given refunds,Neutral
34901,35480,BUSINESS,DON'T PANIC! That Bacon Shortage You Keep Hearing About Isn't Real,Ed Mazza,https://www.huffingtonpost.com/entry/no-bacon-shortage_us_5892dfbae4b0bf5206e63393,america isnt running bacon,2017-02-02,america running bacon,Neutral
57004,58878,BUSINESS,One Map That Shows How The Middle Class Is Getting Squeezed,Shane Ferro,https://www.huffingtonpost.com/entry/us-middle-class-squeezed_us_5734ea76e4b060aa7819c3c6,people arent earning,2016-05-13,people arent earning,Neutral
177642,200136,BUSINESS,Facebook Is No Longer Cool,"Max Kolonko, Contributor",null,null,null,null,null
New York television producer,director,"author and the U.S. te...""",https://www.huffingtonpost.com/entry/facebook-public_us_5bb2b460e4b0480ca659ddfa,facebook billionaire mark zuckerberg joined establishment corporate giant cheated luring concept global friendship turned lined corporate greed,2012-02-05,facebook billionaire mark zuckerberg joined establishment corporate giant cheated luring concept global friendship turned lined corporate greed,Negative,null,null
158327,180430,MONEY,Master the Five Factors That Feed FICO This Fall,"Jeanne Kelly, Contributor",null,null,null,null,null
Credit & Identity Theft Expert,Author,"CEO of The Kelly Gro...""",https://www.huffingtonpost.com/entry/master-the-five-factors-t_us_5b9c372ce4b03a1dcc7d278f,credit score taken summer vacation time commitments changes,2012-09-04,credit score summer vacation time commitments changes,Positive,null,null


In [5]:
file_uri = "/mnt/mnt_s3/BusinessTestData.csv"

testDataset = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(file_uri)
display(testDataset)

_c0,index,category,headline,authors,link,short_description,date,short_description_new,label
106103,127278,MONEY,Future Shock: Personal Finance Technology,"Jim Gibson, Contributor",null,null,null,null,null
Personal Finance writer,"Your Finances Simplified""",https://www.huffingtonpost.com/entry/future-shockpersonal-fina_us_5b9df118e4b03a1dcc8f2e62,land way banking receipts growing trend alternative payment,2014-03-24,land banking receipts growing trend alternative payment,Positive,null,null,null
84389,87560,BUSINESS,City Of Charleston Steps Up To Help Victims' Families In A Big Way,Andrew Lord,https://www.huffingtonpost.com/entry/business-charleston-victims_n_7644912.html,donations pouring local businesses,2015-06-23,donations pouring local businesses,Neutral
116537,137877,BUSINESS,The Shoestring Guide to Brand Publishing,null,https://www.huffingtonpost.com/entry/brand-publishing-budget_us_5bb302d8e4b0480ca66147cf,contently creating stream content stands crowd brand map brand key,2013-11-30,contently creating stream content stands crowd brand map brand key,Positive
60976,63051,BUSINESS,One Of Ben Carson's Craziest Ideas Is Coming True,Ben Walsh,https://www.huffingtonpost.com/entry/us-tax-haven_us_56f56a7be4b014d3fe22f892,worlds hottest new tax haven,2016-03-26,worlds hottest new tax haven,Neutral
174267,196702,MONEY,St. Patrick's Day 2012: Saturday Holiday A Boon For Bars And Retailers,null,https://www.huffingtonpost.com/entry/st-patricks-day-spending_us_5b9b7184e4b03a1dcc779415,celebrating day likely entail spending money st patricks day far expensive holiday,2012-03-13,celebrating day likely entail spending money patricks day far expensive holiday,Positive
92267,101497,BUSINESS,'Breastaurants' Thrive As The Restaurant Industry Struggles,Jillian Berman,https://www.huffingtonpost.com/entry/breastaurants-growth_n_6443274.html,serve burger cleavage crowds come pouring,2015-01-13,serve burger cleavage crowds come pouring,Neutral
97355,111221,BUSINESS,What Should I Do if an Employee Is a Liar?,"Russ Warner, ContributorVP Marketing at Converus, makers of EyeDetect, innovative solu...",https://www.huffingtonpost.com/entry/what-should-i-do-if-an-em_b_5864410.html,need careful assume individuals guaranteed repeat past behavior assumptions limit persons ability learn grow know employee going repeat past behavior illicit drug use theft bribery,2014-09-23,need careful assume individuals guaranteed repeat past behavior assumptions limit persons ability learn grow employee going repeat past behavior illicit drug use theft bribery,Positive
54038,55728,BUSINESS,IEX Wins SEC Approval As U.S. Stock Exchange,null,https://www.huffingtonpost.com/entry/iex-sec-stock_us_576499dee4b015db1bc98569,builtin delay protect investors traders use tactics depicted michael lewis book flash boys wall street revolt,2016-06-18,builtin delay protect investors traders use tactics depicted michael lewis book flash boys wall street revolt,Positive
135872,157630,BUSINESS,"Mohammed Sohel Rana, Bangladesh Factory Owner, Faces Murder Complaint As Death Toll Rises To 622","Reuters, Reuters",https://www.huffingtonpost.com/entry/mohammed-sohel-rana-murder_us_5bb2f3b6e4b0480ca65f6045,guilty killings highest punishment capital punishment said abdul huq,2013-05-05,guilty killings highest punishment capital punishment said abdul huq,Negative


In [6]:
# testDataset.toPandas()

In [7]:
trainDataset.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5666 entries, 0 to 5665
Data columns (total 10 columns):
_c0 5666 non-null object
index 5666 non-null object
category 5666 non-null object
headline 5666 non-null object
authors 4986 non-null object
link 4737 non-null object
short_description 4297 non-null object
date 3930 non-null object
short_description_new 3830 non-null object
label 3807 non-null object
dtypes: object(10)
memory usage: 442.7+ KB

In [8]:
trainDataset = trainDataset.na.drop()

In [9]:
trainDataset= trainDataset.filter((trainDataset["label"]=="Positive") | (trainDataset["label"]=="Negative") | (trainDataset["label"]=="Neutral"))


In [10]:
trainDataset.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118 entries, 0 to 3117
Data columns (total 10 columns):
_c0 3118 non-null object
index 3118 non-null object
category 3118 non-null object
headline 3118 non-null object
authors 3118 non-null object
link 3118 non-null object
short_description 3118 non-null object
date 3118 non-null object
short_description_new 3118 non-null object
label 3118 non-null object
dtypes: object(10)
memory usage: 243.7+ KB

In [11]:
trainDataset.toPandas()['label'].value_counts()

Out[11]: Positive 1483
Negative 843
Neutral 792
Name: label, dtype: int64

In [12]:
trainDataset.show()

+------+------+--------+--------------------+--------------------+--------------------+--------------------+----------+---------------------+--------+
 _c0| index|category| headline| authors| link| short_description| date|short_description_new| label|
+------+------+--------+--------------------+--------------------+--------------------+--------------------+----------+---------------------+--------+
148341|170325|BUSINESS|Stores Open On Ch...| Caroline Fairchild|https://www.huffi...|leave christmas s...|2012-12-22| leave christmas s...|Negative|
 94803|106274|BUSINESS|Trusting the Crow...|Despina Katsikaki...|https://www.huffi...|business environm...|2014-11-19| business environm...|Positive|
 86152| 90632|BUSINESS|"The Sudden Implo...|Tony Potts, Contr...|https://www.huffi...|look secret timel...|2015-05-19| look secret timel...|Negative|
 40379| 41278|BUSINESS|He's Grounded! De...| Carla Herreria|https://www.huffi...|ceo says passenge...|2016-11-29| ceo says passenge...| Neutral|
 34901| 35480|BUSINESS|DON'T PANIC! That...| Ed Mazza|https://www.huffi...|america isnt runn...|2017-02-02| america running ...| Neutral|
 57004| 58878|BUSINESS|One Map That Show...| Shane Ferro|https://www.huffi...|people arent earning|2016-05-13| people arent earning| Neutral|
 88041| 93760|BUSINESS|The High Cost of ...|Leo W. Gerard, Co...|https://www.huffi...|raises course cos...|2015-04-13| raises course cos...|Negative|
135953|157711|BUSINESS|Bangladesh Asks E...| Reuters, Reuters|https://www.huffi...|adds ruma paul se...|2013-05-04| adds ruma paul se...| Neutral|
150904|172914|BUSINESS|Anti-Austerity Ma...| Reuters, Reuters|https://www.huffi...|students occupied...|2012-11-24| students occupied...|Negative|
 64654| 66880|BUSINESS|Martin Shkreli Wa...| Sebastian Murdock|https://www.huffi...|horrible person h...|2016-02-11| horrible person h...|Negative|
129466|151110|BUSINESS|Expected Decline ...| Reuters, Reuters|https://www.huffi...|july reuters unex...|2013-07-13| july reuters unex...|Positive|
102534|122343|BUSINESS|The Potential Pro...|Dan Solin, Contri...|https://www.huffi...|case poses dilemm...|2014-05-20| case poses dilemm...|Positive|
 92274|101505|BUSINESS|4 Ways the Free M...|Patrick Donohoe, ...|https://www.huffi...|human genius come...|2015-01-13| human genius come...|Positive|
100327|117464|BUSINESS|Leader-as-Perform...|Dr. Jim Taylor, C...|https://www.huffi...|performance vital...|2014-07-14| performance vital...|Positive|
164053|186293| MONEY|Consumer Debt Del...| Reuters, Reuters|https://www.huffi...|washington july r...|2012-07-03| washington july r...|Positive|
 83958| 86856|BUSINESS|You Don't Need Ro...|"Valeh Nazemoff, ...|https://www.huffi...|look people like ...|2015-07-01| look people like ...|Positive|
 57872| 59788|BUSINESS|Airline Profits S...| Chris D'Angelo|https://www.huffi...| holy baggage fees|2016-05-03| holy baggage fees| Neutral|
 77222| 79814|BUSINESS|How To Deal With ...|Molly Owens, Cont...|https://www.huffi...|biggest challenge...|2015-09-19| biggest challenge...|Positive|
 65212| 67453|BUSINESS|Obama Ridiculed F...| Shahien Nasiripour|https://www.huffi...|education departm...|2016-02-05| education departm...|Negative|
126245|147817|BUSINESS|States Profiting ...|24/7 Wall St., 24...|https://www.huffi...|like tax foundati...|2013-08-17| like tax foundati...|Positive|
+------+------+--------+--------------------+--------------------+--------------------+--------------------+----------+---------------------+--------+
only showing top 20 rows

In [13]:
testDataset = testDataset.na.drop()

In [14]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline

sparknlp.start()

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [15]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("short_description_new")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
sentimentdl = SentimentDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("label")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        sentimentdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [16]:
trainDataset

Out[16]: DataFrame[_c0: string, index: string, category: string, headline: string, authors: string, link: string, short_description: string, date: string, short_description_new: string, label: string]

In [17]:
trainDataset.filter("short_description_new IS NULL").count()

Out[17]: 0

In [18]:

pipelineModel = pipeline.fit(trainDataset.filter("label IN ('Positive','Negative')"))


In [19]:
predictions =pipelineModel.transform(testDataset.filter("label IN ('Positive','Negative')"))
display(predictions)


_c0 index category headline authors link short_description date short_description_new label document sentence_embeddings class 97355 111221 BUSINESS What Should I Do if an Employee Is a Liar? Russ Warner, ContributorVP Marketing at Converus, makers of EyeDetect, innovative solu... https://www.huffingtonpost.com/entry/what-should-i-do-if-an-em_b_5864410.html need careful assume individuals guaranteed repeat past behavior assumptions limit persons ability learn grow know employee going repeat past behavior illicit drug use theft bribery 2014-09-23 need careful assume individuals guaranteed repeat past behavior assumptions limit persons ability learn grow employee going repeat past behavior illicit drug use theft bribery Positive List(List(document, 0, 175, need careful assume individuals guaranteed repeat past behavior assumptions limit persons ability learn grow employee going repeat past behavior illicit drug use theft bribery, Map(sentence -> 0), List())) List(List(sentence_embeddings, 0, 175, need careful assume individuals guaranteed repeat past behavior assumptions limit persons ability learn grow employee going repeat past behavior illicit drug use theft bribery, Map(sentence -> 0, token -> need careful assume individuals guaranteed repeat past behavior assumptions limit persons ability learn grow employee going repeat past behavior illicit drug use theft bribery, pieceId -> -1, isWordStart -> true), List(-0.026753519, -0.06613543, 0.021957638, -0.037745513, -0.0143613275, -0.061664544, 0.06308372, 0.0742839, -0.017537989, -0.019150887, 0.02965685, 0.019790825, -0.029754845, 0.045549203, 0.048701644, 0.005113714, 0.021654988, -0.026466798, -0.0096040685, -0.06573076, -0.05758304, -0.057155076, 0.023634197, 0.07851815, -0.010818237, 0.054342404, 0.016012575, -0.0050820815, 0.051100656, 0.07036361, -0.006677968, 0.0051496075, 0.017857088, -0.06283593, -0.056294624, 0.07632955, -0.006377195, -0.010537326, -0.0023182598, -0.07434983, -0.047324408, -0.08110785, 0.062293895, 0.03412874, -0.072305225, 0.03166609, -0.044721287, 0.050257616, 0.039364316, -0.060328916, 8.093875E-4, -0.06974288, -0.0069326907, 0.0819661, 0.029750576, 0.035343442, 0.014395629, -0.05725947, 0.028268315, -0.001666052, 0.017499419, -0.009351831, -0.024730466, 0.06545373, -0.050353393, -0.050055385, 0.04455707, 0.01189179, -0.06860811, 0.0786158, -0.016739568, -0.064580955, -0.055565104, 0.054989632, -0.056830943, 0.07027246, 0.012856143, 0.05394035, -0.06846935, 0.008131751, -0.0120494645, 0.04661547, 5.6413835E-4, 0.06400421, -0.02841225, -0.027347928, 0.033231683, -0.039954156, 0.05196822, -0.045936186, 0.057092596, 0.0568143, 0.007966548, -0.013911997, 0.04418936, -0.07994143, -0.0035121604, 0.008998999, -0.032881074, -0.042186264, 0.05084107, -0.015498984, -0.006696876, 0.021666773, -0.018061018, -0.008862957, 0.022089172, -0.022070913, -0.04776562, 0.065761715, -0.04799857, -0.0015079343, -0.023672799, 0.053676337, -0.07694522, -0.030932523, -0.034452863, -0.043033775, 0.074225254, 0.0069878353, -0.04994391, -0.046872653, 0.0011246318, 0.020778816, 0.0055921315, -0.026367186, 0.015109871, -0.076251954, -0.049522918, 0.049931925, 0.041559793, 0.05632196, -0.0027419303, 0.07040185, 0.031044515, -0.034742028, 0.05975918, -0.009076552, 0.029156797, -0.031882405, 0.049819954, -0.0014379157, -0.0643176, 0.030386921, 0.026839213, 0.06158101, -0.066608846, -0.059250187, 0.075724415, 0.06214568, -0.03275073, -0.027772153, 0.0020310676, 0.037126567, 0.0819702, 0.006849605, -0.043387122, 0.034192357, 0.012243119, 0.050462775, -0.020519206, 0.054391928, 0.07757439, 0.055825725, -0.046602216, 0.008672626, 0.029312233, 0.03430933, -0.0047544613, -0.052360557, 0.027083712, 0.0024792056, -0.012250074, -0.004637208, -0.028171843, -0.02183232, -0.008813422, 0.009373757, -0.050734635, 0.059804138, 0.009018574, 0.082975395, -0.07631025, -0.024672488, 0.028052865, -0.013731058, 0.04941563, -0.021041246, 0.0764508, 0.030558044, -0.0031345196, 0.06440952, -0.012626311, 0

In [20]:
new_predictions = predictions.selectExpr("*", "class.result[0] as prediction")
display(new_predictions.select("label", "prediction"))

label,prediction
Positive,Positive
Negative,Negative
Negative,Negative
Positive,Positive
Positive,Positive
Positive,Positive
Positive,Positive
Positive,Negative
Positive,Positive
Positive,Positive


In [21]:
@F.udf(returnType="double")
def convert_label(label):
  if label == "Positive":
    return 1.0
  if label == "Negative":
    return -1.0
  return 0.0

new_new_predictions = new_predictions \
                    .withColumn("label_double", convert_label("label")) \
                    .withColumn("prediction_double", convert_label("prediction"))
display(new_new_predictions.select("label_double", "prediction_double"))

label_double,prediction_double
1.0,1.0
-1.0,-1.0
-1.0,-1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,-1.0
1.0,1.0
1.0,1.0


In [22]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_double", predictionCol="prediction_double",                                                  metricName="accuracy")
evaluator.evaluate(new_new_predictions)

Out[45]: 0.761252446183953

In [23]:
predictions.printSchema()

root
-- _c0: string (nullable = true)
-- index: string (nullable = true)
-- category: string (nullable = true)
-- headline: string (nullable = true)
-- authors: string (nullable = true)
-- link: string (nullable = true)
-- short_description: string (nullable = true)
-- date: string (nullable = true)
-- short_description_new: string (nullable = true)
-- label: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence_embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- class: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [24]:
predictions = predictions.selectExpr("*", "class.result[0] as prediction")
display(predictions)

_c0 index category headline authors link short_description date short_description_new label document sentence_embeddings class prediction prediction 84389 87560 BUSINESS City Of Charleston Steps Up To Help Victims' Families In A Big Way Andrew Lord https://www.huffingtonpost.com/entry/business-charleston-victims_n_7644912.html donations pouring local businesses 2015-06-23 donations pouring local businesses Neutral List(List(document, 0, 33, donations pouring local businesses, Map(sentence -> 0), List())) List(List(sentence_embeddings, 0, 33, donations pouring local businesses, Map(sentence -> 0, token -> donations pouring local businesses, pieceId -> -1, isWordStart -> true), List(-0.02514095, 0.050500564, 0.0016553696, 0.05828257, 0.03125392, -0.043511942, 0.047940336, 0.00830307, -0.023821373, -0.0396505, -0.0012316467, -0.031058405, -0.03438056, -0.04548464, 0.046468638, -0.03600206, 0.0687863, -0.0110314125, 0.059868447, 0.04060255, 0.026801642, -0.07712967, 0.029087696, 0.0384851, 0.013989238, 0.032787953, 0.045423836, -0.051678084, 0.035805095, -0.045095235, 0.042619362, -0.002948102, 2.0364934E-4, 0.054669324, -0.05385396, -0.026279034, 0.041756444, 0.04795833, 0.0277866, -0.060664922, -0.010551428, -0.08158213, 0.025074365, 0.0127950525, -0.054438233, 0.036288906, -0.03449274, -0.06785424, 0.0058760582, -0.050704565, 0.06523468, -0.058416635, 0.0038709443, 0.07681574, 0.05055782, 0.009694006, 0.069831, -0.04661248, 0.002245499, -0.03919958, -0.057033617, -0.021769311, -0.0025431577, -0.07910321, 0.028148104, 0.008441109, 0.025860574, 0.04951387, 0.07668092, 0.054011364, 0.03243865, -0.008517213, 0.064122945, 0.04353457, -0.0418931, 0.07945231, 0.029885925, 0.074187264, 0.0056260703, -0.032980938, 0.030561026, 0.022518221, 0.021560099, 0.01348767, 0.036608003, 0.012584343, 0.009281538, -0.018393187, -0.047333106, -0.013478538, 0.059451576, 0.013022759, 0.016990611, -0.045541275, 0.058844615, -0.078573726, 0.031686824, 0.011987308, -0.023422217, 0.070194915, 0.06985528, -0.034294445, -0.06627991, 0.038059913, 0.0012427055, 0.003530947, 0.056164682, 0.00560785, -0.01598631, 0.06788596, 0.025309954, -0.027165642, 0.039242845, 0.06735746, -0.0020333384, 0.075596966, -0.058952913, -0.046941046, 0.043139275, 0.04069216, 0.062571295, -0.007291711, -0.026278904, -0.011669707, 0.007733346, -0.04876303, 0.01430403, -0.06166214, -0.022617178, 0.060264587, -0.004221452, 0.068092056, 0.021994555, 0.06379712, -0.00755331, 0.032417856, -0.0015145751, 0.039294533, 0.054188114, 0.06402971, -0.032388043, -0.053392533, -0.049079075, -0.04060449, 0.019519228, -0.07112947, -0.050875187, 0.044142466, -0.022956504, -0.01099947, 0.061566878, -0.06096938, 0.024862632, -0.06614169, 0.048565138, -0.03792578, 0.047185604, 0.0034395126, 0.050869685, 0.04313697, -0.030436577, -0.064983785, 0.013924746, 0.077386364, 0.04516596, 0.07678586, 0.07766729, 0.065719664, 0.025681442, 0.011678189, -0.056290355, 0.011295996, -0.05738683, 0.05796317, -0.04329911, 0.009217143, -0.038275607, -0.025127213, -0.05506115, -0.050719008, -0.051259417, 0.013680324, 0.057439767, 0.02973081, -0.0743916, -0.036532227, -0.017839244, -0.06256046, 0.0052656992, -0.0018596094, 0.037531555, 0.067500174, 0.020022163, 0.03876975, -0.053852983, -0.021884779, 0.047776457, 0.075181715, 0.07701731, 0.022102052, -0.010741394, 0.026668714, 0.014072525, 0.047880173, 0.011935849, 0.029521652, 0.050718978, -0.053864952, -0.010683063, 0.04612589, 0.04362773, -0.01715941, -0.068211466, 0.031242272, 0.081973314, 0.031985797, -0.042147126, -0.05254794, 0.0032525854, 0.07245983, 0.056927763, -0.015573962, 0.016039312, -0.074241474, 0.0052414206, -0.04610818, 0.0022622363, -0.008245154, 0.065927856, -3.6553937E-4, 7.511077E-4, -0.027051609, 0.052594613, -0.039007004, -0.0539665, 0.028320033, 0.040739328, -0.03941588, 0.0711225, 0.008200817, 0.042259887, 0.03845848, 0.029991655, 0.033027887, 0.034915086, -0.0686931, 0.06693859, -0.061579756, 0.01172007, -0.059116285, -0.06560866, -0.017774884, 

In [25]:
pipelineModel.save("/mnt/mnt_s3/sentimentdl_pipeline_2")

In [26]:
loadedPipeline = PipelineModel.load("/mnt/mnt_s3/sentimentdl_pipeline_2")

In [27]:
import pandas as pd
pdf = pd.DataFrame({"text": [
  "This is so great! How are you", 
  "No way, never again!", 
  "Yes, bring on more!"
]})
df = spark.createDataFrame(pdf)

results = pipelineModel.transform(df)
display(
  results
)

In [28]:
display(
  results.selectExpr("text", "class.result")
)

In [29]:
analyzer.polarity_scores("This is so great! How are you")

In [30]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

@F.udf(returnType="struct<neg:double,neu:double,pos:double,compound:double>")
def getVADER(text):
	return analyzer.polarity_scores(text)

new_results = results.withColumn("vaderSentiment", getVADER("text"))
display(new_results)